In [ ]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
!pip install split_folders

In [ ]:
import split_folders

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

In [ ]:
os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')

In [ ]:
!ls ../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian

In [ ]:
img_loc = '../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

split_folders.ratio(img_loc, output='output', seed=1, ratio=(0.8, 0.2))

In [ ]:
!ls output

In [ ]:
train_loc = 'output/train/'
test_loc = 'output/val/'

In [ ]:
trdata = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224))
tsdata = ImageDataGenerator(rescale = 1./255)
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224))

In [ ]:
droprate=0.1
filter_pixel=3
input_shape = (224, 224, 3)
classifier = Sequential()

In [ ]:
#convolution 1st layer
classifier.add(Convolution2D(32, kernel_size=(filter_pixel, filter_pixel), input_shape = input_shape, activation = 'relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#convolution 2nd layer
classifier.add(Convolution2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#convolution 3rd layer
classifier.add(Convolution2D(128, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

classifier.add(Convolution2D(256, kernel_size=(filter_pixel, filter_pixel), activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(droprate))

#Fully connected 1st layer
classifier.add(Flatten()) 
classifier.add(Dense(4048)) 
#classifier.add(BatchNormalization())
classifier.add(Activation('relu')) 
#classifier.add(Dropout(droprate))

#Fully connected 1st layer
classifier.add(Dense(4048)) 
classifier.add(Activation('relu')) 
classifier.add(Dense(2024)) 
classifier.add(Activation('relu')) 

#Fully connected final layer
classifier.add(Dense(5))
classifier.add(Activation('softmax'))

In [ ]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_loc,
                                                 target_size = (224, 224),
                                                 batch_size = 32)

test_set = test_datagen.flow_from_directory(test_loc,
                                            target_size = (224, 224),
                                            batch_size = 32)

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch=training_set.samples//training_set.batch_size, 
                         validation_data=test_set,
                         validation_steps=test_set.samples//test_set.batch_size,
                         epochs=20)
classifier.save('DR_CNN.m5')

In [ ]:
score = classifier.evaluate(test_set, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])